## Constants

In [11]:
import os

file_count = 50   # TODO: change to 1000
playlists_per_file = 1000

dir_data_raw = os.path.join("..", "data", "raw")
dir_data_processed = os.path.join("..", "data", "processed")

playlist_count = file_count * playlists_per_file

Merge the data extracted from the Spotify API (csv file) with that of the Spotify Million Playlist Dataset (pickle file)

In [41]:
import os
import pandas as pd

path_audio_features = os.path.join(dir_data_raw, "tracks", "track_audio_features.csv")
audio_features = pd.read_csv(path_audio_features)

display(audio_features)

,id,tempo,key,mode,loudness,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,time_signature
0,3zyYVItCMCjFzBHTyjrxPK,79.227,8.0,1.0,-3.406,0.466,0.856,0.3180,0.257000,0.000000,0.6750,0.531,292223.0,4.0
1,4jSy0HTIoC9yiwZ8OVyTCW,164.207,1.0,1.0,-7.972,0.510,0.849,0.1190,0.000546,0.000748,0.6890,0.887,315067.0,4.0
2,4zyqBSUFNkJ20mw1FB68gt,83.947,4.0,0.0,-22.867,0.308,0.114,0.0321,0.958000,0.902000,0.0853,0.303,350906.0,4.0
3,63B3TtwUzOoJoe3unMteVa,93.696,1.0,0.0,-4.166,0.660,0.943,0.2770,0.129000,0.000000,0.5570,0.599,210733.0,4.0
4,7y9iMe8SOB6z3NoHE2OfXl,118.384,0.0,1.0,-3.539,0.675,0.751,0.0296,0.060400,0.000000,0.0893,0.612,181279.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38144,0s9iofqbQdsmWZQJ1TZh1c,118.061,7.0,1.0,-13.766,0.550,0.351,0.0443,0.531000,0.006020,0.1500,0.237,162826.0,4.0
38145,5Ozvg38CnCQKmPP8tgSRA6,96.520,0.0,1.0,-4.130,0.609,0.742,0.0477,0.001310,0.000002,0.0725,0.531,256227.0,4.0
38146,2yWsZpLZNmap92YkGkzyDq,179.985,6.0,1.0,-5.899,0.373,0.584,0.0272,0.004810,0.449000,0.1630,0.403,214667.0,3.0
38147,7JxA6bPIyXNohnrTw6CzS0,178.012,7.0,0.0,-7.096,0.473,0.829,0.3620,0.048900,0.000000,0.2530,0.583,270680.0,4.0


Create a function to compute relevant metrics about a playlist's audio features.

In [236]:
def get_playlist_metrics(df: pd.DataFrame):
    def get_statistic(f):
        target_columns = list(range(2,9)) # TODO: which columns do we care about?
        return df.iloc[:, target_columns].apply(f).to_frame().transpose()

    return pd.concat({"Mean" : get_statistic(pd.Series.mean),
                      "Median" : get_statistic(pd.Series.median),
                      "Standard deviation" : get_statistic(pd.Series.std),
                      "Variance" : get_statistic(pd.Series.var)}, axis=1)

Combine extracted and calculated data into a playlist frame

In [237]:
import pickle

playlist_list_path = os.path.join(dir_data_processed, "playlists" + str(playlist_count) + ".pkl")

with open(playlist_list_path, "rb") as fin:
    playlist_data = pickle.load(fin)

playlists_metrics = pd.DataFrame()

for playlist in playlist_data[2:10]:
    # print("Number of tracks: {}".format(len(playlist["track_ids"])))    
    playlist_features = pd.DataFrame(columns = audio_features.columns.tolist())

    for track_id in playlist["track_ids"]:
        track_features = audio_features[audio_features["id"] == track_id]
        
        if track_features.empty: # TODO: handle cases when track id has no audio features
            # print("{} has no audio features".format(track_id))
            continue

        playlist_features = pd.concat([playlist_features if not playlist_features.empty else None, 
                                       track_features])
        
    playlist_info = pd.DataFrame(data={k:[v] for k,v in playlist.items() if k != "track_ids"})
    playlist_metrics = pd.concat([playlist_info, get_playlist_metrics(playlist_features)], axis=1)
    playlists_metrics = pd.concat([playlists_metrics, playlist_metrics], ignore_index=True)

display(playlists_metrics)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,num_edits,num_artists,"(Mean, key)",...,"(Standard deviation, energy)","(Standard deviation, speechiness)","(Standard deviation, acousticness)","(Variance, key)","(Variance, mode)","(Variance, loudness)","(Variance, danceability)","(Variance, energy)","(Variance, speechiness)","(Variance, acousticness)"
0,abby,false,10,1509321600,72,60,2,36,40,7.000000,...,0.158421,0.131639,0.244839,1.200000,0.166667,10.290011,0.016980,0.025097,0.017329,0.059946
1,mixtape,false,20,1509494400,14,9,3,11,6,2.500000,...,0.147785,0.123744,0.021850,4.500000,0.000000,0.120541,0.000005,0.021841,0.015313,0.000477
2,fall '17,false,22,1509408000,42,39,2,15,37,3.500000,...,0.180312,0.081034,0.032527,4.500000,0.500000,0.051200,0.001458,0.032512,0.006567,0.001058
3,run it,false,25,1488412800,41,38,2,8,31,4.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Yeet,false,28,1509062400,81,57,7,26,37,4.416667,...,0.199737,0.110656,0.213799,12.265152,0.272727,9.187679,0.012511,0.039895,0.012245,0.045710
5,2020,false,40,1490659200,85,80,6,27,71,3.666667,...,0.179101,0.137868,0.079118,12.333333,0.333333,0.281064,0.009164,0.032077,0.019008,0.006260
6,Road Trippin',false,42,1490572800,158,131,2,51,106,3.500000,...,0.175783,0.017871,0.203841,12.277778,0.233333,1.432527,0.011403,0.030900,0.000319,0.041551
7,taylor swift,false,44,1430611200,33,29,3,18,21,2.800000,...,0.178884,0.087353,0.216835,12.700000,0.000000,1.446937,0.063254,0.031999,0.007631,0.047017


In [ ]:
import pickle

playlist_list_path = os.path.join(dir_data_processed, "playlists" + str(playlist_count) + ".pkl")

with open(playlist_list_path, "rb") as fin:
    playlist_data = pickle.load(fin)

playlist_frame
playlist_frame = pd.DataFrame(np.arrange())